In [1]:
from models.mlp import MLP
from dataloader import get_dataset, get_avg_dataloader
import os
import torch
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [2]:
dataX_path = os.path.join(os.path.join('data/processed/mean_ACE.pkl'))
dataY_path = os.path.join(os.path.join('data/processed/ACE_Y_mean.pkl'))
dataX, dataY = get_dataset(dataX_path,dataY_path)
dataloaders = get_avg_dataloader(dataX,dataY, batchsize=128)

args = {
        'name':'Updated_data',
        'trainer':{
            'save_dir':'save',
            'epochs': 2000,
            'lr':10e-3
        },
        'model':{
          'type':'MLP',
          'input_size':11,
          'output_size':1,
          'hidden_layers':[1024,512,256,64],
    
        }
    }


In [3]:
model = MLP(input=11, output=1, hidden_layers=[1024,512,256,64]).to('cuda')
state = torch.load('save/model/Updated_data/mlp/model.best.pth')
model.load_state_dict(state['state_dict'])

<All keys matched successfully>

In [4]:
def adjusted_r2_score(y_true, y_pred, n_features):
    r2 = r2_score(y_true, y_pred)
    n_samples = len(y_true)
    adjusted_r2 = 1 - (1 - r2) * (n_samples - 1) / (n_samples - n_features )
    return adjusted_r2
def test(net,test_loader,device):
    y_pred = []
    y_true = []
    net.train(False)
    net.eval()
    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device), targets
        y_pred.extend(net(inputs).cpu().data.numpy())
        y_true.extend(targets.numpy())

    print("MAE:", mean_absolute_error(y_true, y_pred))
    print("RMSE:", mean_squared_error(y_true, y_pred,squared=False))
    print("R^2:", r2_score(y_true, y_pred))
    print('adjusted r2',adjusted_r2_score(y_true, y_pred, 11))

In [5]:
test(model, dataloaders['test_loader'],device='cuda')

MAE: 1.0946486
RMSE: 2.2452648
R^2: 0.9669360818114536
adjusted r2 0.9668865775171154


In [7]:
min(state['training_losses']),min(state['validation_losses'])

(4.833022814850475, 4.196104887369517)